In [1]:
import speech_recognition as sr
from os import path
import subprocess
import sys
def speech_recognizer(path,file,start=0.0,end=0):
    try:
        os.remove(path+"/videos/out.wav")
    except OSError:
        pass

    filename = file.split('.')[0]
    if(end>0):
        start = start - 1.0
        command = "ffmpeg -i "+ path+"/videos/"+file+" -ss "+str(start)+" -to "+str(end)+" -acodec copy "+path+"/videos/out.wav"
        subprocess.call(command, shell=True)
        AUDIO_FILE =  "out.wav"
    elif end == 0:
        AUDIO_FILE =  file       
    r = sr.Recognizer()
    with sr.AudioFile(path+"/videos/"+AUDIO_FILE) as source:
        r.adjust_for_ambient_noise(source)
        audio = r.record(source)  # read the entire audio file

    try:
        r = r.recognize_sphinx(audio)
        print r
        return r
    except sr.UnknownValueError:
       # os.remove("out.wav")
        print("Sphinx could not understand audio")
        return 
    except sr.RequestError as e:
       # os.remove("out.wav")
        print("Sphinx error; {0}".format(e))
        return


def extract_audio(filename, channels=1, rate=44100):
    temp = tempfile.NamedTemporaryFile(suffix='.wav', delete=False)
    if not os.path.isfile(filename):
        print ("The given file does not exist: {0}".format(filename))
        raise Exception("Invalid filepath: {0}".format(filename))
    if not which("ffmpeg"):
        print ("ffmpeg: Executable not found on machine.")
        raise Exception("Dependency not found: ffmpeg")
    command = ["ffmpeg", "-y", "-i", filename, "-ac", str(channels), "-ar", str(rate), "-loglevel", "error", temp.name]
    use_shell = True if os.name == "nt" else False
    subprocess.check_output(command, stdin=open(os.devnull), shell=use_shell)
    return temp.name, rate





In [2]:

def percentile(arr, percent):
    arr = sorted(arr)
    k = (len(arr) - 1) * percent
    f = math.floor(k)
    c = math.ceil(k)
    if f == c: return arr[int(k)]
    d0 = arr[int(f)] * (c - k)
    d1 = arr[int(c)] * (k - f)
    return d0 + d1



In [3]:
import subprocess, sys
from pytube import YouTube
import pysrt
import os
import shutil
import requests
from __future__ import absolute_import, print_function, unicode_literals
import argparse
import audioop
import json
import math
import multiprocessing
import os
import requests
import subprocess
import sys
import tempfile
import wave

path = os.getcwd()
if not os.path.exists(os.path.dirname(path+"/videos/")):
    os.makedirs(os.path.dirname(path+"/videos_text/"))

if not os.path.exists(os.path.dirname(path+"/videos/")):
    os.makedirs(os.path.dirname(path+"/videos_text/"))

def autosub_func(path,name,srt,lan):     
    command = "python /Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/autosub/autosub_app.py -S en -D en  "+path+"/videos/"+name
    subprocess.call(command, shell=True)
    srt = srt.replace("_"+lan,"")
    print("!!!!!!!!!!")
    print (srt)
    shutil.move(path+"/videos/"+srt, path+"/videos_text/"+srt.split('.')[0].strip()+"_"+lan+".srt")

def cmu(path,file,srt):
#    filename ,rate = extract_audio(path+"/videos/"+file)
    print (srt)
    filename = srt.split('.srt')[0].strip()+".wav"
    print (filename,file)
    command = "ffmpeg -i "+path+"/videos/"+file+" -ab 160k -ac 2 -ar 44100 -vn "+path+"/videos/"+filename
    subprocess.call(command, shell=True)
    regions = find_speech_regions(path+"/videos/"+filename)
    with open(path+"/videos_text/"+srt,"w+") as writer:
        for i in range(0,len(regions)):
            recognizer = speech_recognizer(path,filename,regions[i][0],regions[i][1])
            writer.write("%s\n"%str(i+1))                
            sec, micro = str(regions[i][0]).split('.')
            m, s = divmod(int(sec), 60)
            h, m = divmod(m, 60)
            time= "{:02}:{:02}:{:02},{}".format(h,m,s,micro)
            writer.write("%s"%time)        
            writer.write(" --> ")
            sec, micro = str(regions[i][1]).split('.')
            m, s = divmod(int(sec), 60)
            h, m = divmod(m, 60)
            time1= "{:02}:{:02}:{:02},{}".format(h,m,s,micro)
            writer.write("%s"%time1)
            writer.write("\n")
            writer.write("%s"%recognizer)
            writer.write("\n\n")

    

def download_file(path,url):
    vid_form = [".3G2",".3GP",".ASF",".AVI",".FLV",".M4V",".MOV",".MP4",".MPG",".RM",".SRT",".SWF",".VOB",".WMV",".webm"]
    flag=0
    local_filename = url.split('/')[-1]

    for i in vid_form:
        if i.lower() in local_filename:
            flag =1
            x = local_filename.split(i.lower())[0]+i.lower()
    if flag==1:
        r = requests.get(url, stream=True)
        with open(path+"/videos/"+x, 'wb') as f:
            shutil.copyfileobj(r.raw, f)
    else:
        x = local_filename+".mp4"
        r = requests.get(url, stream=True)
        with open(path+"/videos/"+x, 'wb') as f:
            shutil.copyfileobj(r.raw, f)
    return x
def video_to_text(name):
    lan = name[:2]
    name = name[2:]
    
    ################################ Downlaoding from Youtube ##################################
    if "yout" in name:
        out_name =  name.split('/')[-1]+".mp4"
        cmd = "youtube-dl -o "+path+"/videos/"+out_name+" -f mp4 " + name
        subprocess.call(cmd, shell=True)
    ################################ Downlaoding from A Given Video-Link ##################################
    elif "https://" in name:
        out_name = download_file(path,name) 
    ################################ From the local file ##################################
    else:
        try:
            out_name = name.replace(".com","").replace(" ","_")
            print(out_name)
            os.rename(path+"/videos/"+name,path+"/videos/"+out_name)
        except:
            out_name=name
        
    srt = out_name.split('.')[0].strip()+"_"+lan+".srt"  
    print (srt)
    if lan == 'in':
        print ("inside autosub")
        autosub_func(path,out_name,srt,lan)
        return get_srt_to_txt(path,srt)
    if lan == 'us':
        print ("inside cmu")
        #cmu(path,out_name,srt)
   #     filename,rate = extract_audio(path+"/videos/"+out_name)
        filename = srt.split('.srt')[0].strip()+".wav"
        command = "ffmpeg -i "+path+"/videos/"+out_name+" -ab 160k -ac 2 -ar 44100 -vn "+path+"/videos/"+filename
        subprocess.call(command, shell=True)
        return speech_recognizer(path,filename)


def get_srt_to_txt(path,srt):
    sub=""
    subs = pysrt.open(path+"/videos_text/"+srt, encoding='iso-8859-1')
    text = open(path+"/videos_text/"+srt.split('.srt')[0].strip()+".txt","wb+")
    txt = subs[0].text
    sub_len = len(subs)
    for i in range(1,sub_len):
        if ((subs[i].start.minutes*60*1000 + subs[i].start.seconds*1000+subs[i].start.milliseconds) - ((subs[i-1].end.minutes*60*1000 + subs[i-1].end.seconds*1000+subs[i-1].end.milliseconds))) < 500: 
            txt = txt + " "+subs[i].text.lower()
        else:
            txt = txt + ". "
            text.write(txt)
            sub = sub +txt
            txt = subs[i].text
    if sub =="":
        return "No subtitles formed"

    return sub




OSError: [Errno 17] File exists: '/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/videos_text'

In [4]:
text="Kristoff with soldering iron in this video I will be about the six main criteria death make or break form design in quarry. Korean colours. Interface update camera quality battery life and try starting right now infra 17 a few interesting new things happened to fall for bigger and entellus cream. And II will forget very expensive. And the most expensive of them all is the new iPhone 10. Apple's $1,000 iPhone looks. Quizzes for the knot. It is especially important but you get used to that would cut out. That is also a fake Id is on the iPhone 10. The new three face recognition system works surprisingly well in many cases but it is a slower in a bit less accurate. Fingerprint scanner. IPhone has a good looking design screen look gorgeous it is guaranteed to get software update for long time restaurants near me and has a great camera. Battery life is just average stomach virus going to sony iphone 8 plus nios ii has going to notifications my phone i am just rest for the $1,000. And then there be iPhone 8 and iPhone. Suppose you have just call them this 7s and 7s Plus stable comma exactly like last year's iphone. Paytm cashback. Yes the design is a bit boring but it option. Vaibhav improving to main area. Performance with appointments. Is the fastest on any phone emirates how to add yaar really stunning colours. Ambattur wondering the main camera selfie. Beautiful photo as on the iPhone 10. And one more thing the new iphone now supports wireless charging it might be nice but it's slower than what is charging and you cant use your phone was charging we go straight to the land of the free android with the samsung galaxy note 8. That it has one unit t-shirt espn spouses cold enough it was great. Great phone overall has two cameras in shrewsbury good looking for teachers. It has a stylish glass and metal body NS short super good looking screen and solid battery life. Smoke effect no phone is. The know that my baby. Beck so many people coc new 8000 dollars. IPhone and don't expect get android update directly google pixel 2 xl does form ever made basna. The main reason for that is the screen is so matte screen did get it pursuant when is the fox life. NS suffers from Britain an issue with all outlets space very quickly on the pic. How to excel. YouCam perfect screen the pixel 2 XL is it we fall. Buttery smooth performance latest android software jayanti day 1 update and for me a best camera ever put on a phone it's too pricey but cops versus iphone 10 and the note. LG v30 is the best phone LG has ever made. It's our favourite. When it comes piggy bank with basil. The LG v30 has mh12 play almost no bezel no not know cause no interruption. Hey beautiful birth wishes to somebody song with ly dishes with colours this is not is better than the pixel 2 xl but it's too something to know about. Royal digital if interest rate is not perfect text book it is function. If it is very good picture sent know what a fraction form has it super wide angle camera. Ravi Taori also look stylish relation has a baba farid battery life. The oneplus 5t. Paper for the ball flagship phone it is just that for half the price of the policy of just mention. Oneplus 5t delivery similar performance. It has H2H beautiful 6 inch amoled screen has a very fast rashmi performance. It has a stylish metal design in solid battery life. If camera is the only thing that is not buy that girl is another flagship. What is definite too far behind don't forget that the oneplus 5t she is only $4 mv not working hd images like verizon. Springdale United States. Great phone you should not miss is the word made santro vivacious is it wrong to become the world's biggest homemaker. Find Apple and Samsung and the mate 10 pro is is most who is the phone. It is a big phone with a trendy basil extreme stylish glass and metal design islamic colours it has a good looking this play has longer battery life then moves from this place. Now is custom Android interface just take without getting used to super functional and if you want to be different this is a solid choice. There are a few other options keep in mind. Samsung Galaxy S8 Galaxy yes it was her now so that big discount tangled 2 full form premium design in great cameras smart price. The smaller pixel 2 has an amazing camera and doesn't cost proportion was it has no display issues white sibling. Go to excel. Combine this with buttermilk performance and the guarantee for Dev on Android update. And this is our phone worth considering. Finally there is the essential phone. Don't be so for just $450 the essential phone is essentially a steel. It is one of the best designs of 2017 within a split screen in a very compact body scratch resistant ceramic back like no other means she found it is getting software updates on a very regular basis. Is too doesn't have a great camera but if you're willing to deal with a mediocre camera it's a great and very affordable option. These are all favourite phones of 2017 birthday many more worth considering phone by sony show me motorola in many others did not make it in this list define options. So. What is your favourite phone of the year. What is your favourite new feature what is the thing that most love about it."

In [ ]:
video_to_text("usjumanji.mp4")

In [ ]:
text="uswatch?v=Meif6leYjmc.mp4"

In [ ]:
text[2:]

In [ ]:
print (text)

In [ ]:
from pydub import AudioSegment
from pydub.playback import play

In [ ]:
sound_stereo = AudioSegment.from_file("videos/index_us_us.wav", format="mp3")
sound_monoL = sound_stereo.split_to_mono()[0]
sound_monoR = sound_stereo.split_to_mono()[1]


In [ ]:
sound_monoR_inv = sound_monoR.invert_phase()

In [ ]:
sound_CentersOut = sound_monoL.overlay(sound_monoR_inv)

In [ ]:
fh = sound_CentersOut.export("new_file.mp3", format="mp3")

In [ ]:
fh = sound_monoR.export("new_file_R.mp3", format="mp3")

In [ ]:
fh = sound_monoL.export("new_file_L.mp3", format="mp3")

In [ ]:
from scipy.io import wavfile
from matplotlib import pyplot as plt
import numpy as np

# Load the data and calculate the time of each sample
samplerate, data = wavfile.read('videos/index_us_us.wav')
times = np.arange(len(data))/float(samplerate)

# Make the plot
# You can tweak the figsize (width, height) in inches
plt.figure(figsize=(50, 5))
plt.fill_between(times, data[:,0], data[:,1], color='k') 
plt.xlim(times[0], times[-1])
plt.xlabel('time (s)')
plt.ylabel('amplitude')
# You can set the format by changing the extension
# like .pdf, .svg, .eps
#plt.savefig('plot.png', dpi=100)
plt.show()

In [ ]:
sec, micro = 10,4954195011
m, s = divmod(int(sec), 60)
h, m = divmod(m, 60)
time= "{:02}:{:02}:{:02},{}".format(h,m,s,micro)
 

In [ ]:
micro = int(str(micro)[:3])

In [ ]:
(micro)

In [ ]:
import pandas as pd


In [ ]:
x =pd.read_csv("dict.txt")

In [ ]:
import speech_recognition as sr
from os import path
import subprocess
import sys
import subprocess, sys
from pytube import YouTube
import pysrt
import os
import shutil
import requests
from __future__ import absolute_import, print_function, unicode_literals
import argparse
import audioop
import json
import math
import multiprocessing
import os
import requests
import subprocess
import sys
import tempfile
import wave
def find_speech_regions(filename, frame_width=4096, min_region_size=6, max_region_size=15):
    reader = wave.open(filename)
    sample_width = reader.getsampwidth()
    rate = reader.getframerate()
    n_channels = reader.getnchannels()
    chunk_duration = float(frame_width) / rate

    n_chunks = int(math.ceil(reader.getnframes()*1.0 / frame_width))
    energies = []

    for i in range(n_chunks):
        chunk = reader.readframes(frame_width)
        energies.append(audioop.rms(chunk, sample_width * n_channels))

    threshold = percentile(energies, 0.2)

    elapsed_time = 0

    regions = []
    region_start = None
    #print energies
    for energy in energies:
        
        is_silence = energy <= threshold
        max_exceeded = region_start and elapsed_time - region_start >= max_region_size

        if (max_exceeded or is_silence) and region_start:
           # print energy
            if elapsed_time - region_start >= min_region_size:
                regions.append((region_start, elapsed_time))
                #print regions,"region"
                region_start = elapsed_time

        elif (not region_start) and (not is_silence):
            #print elapsed_time,"elasped_start"
            region_start = elapsed_time
            #print region_start,"start"
        elapsed_time += chunk_duration
    return regions
def split_autosub(filename,file):
    cmd = "youtube-dl -o "+filename+" -f mp4 " + file
    subprocess.call(cmd, shell=True)
    print (filename,file)
    wav_file = filename.split('.mp4')[0]+".wav"
    command = "ffmpeg -i "+filename+" -ab 160k -ac 1 -ar 16000 "+wav_file
    subprocess.call(command, shell=True)
    regions = find_speech_regions(wav_file)
    for i in range(0,len(regions)): 
        command = "ffmpeg -i "+ wav_file+" -ss "+str(regions[i][0])+" -to "+str(regions[i][1])+" -acodec copy out"+str(i)+".wav"
        subprocess.call(command, shell=True)
        sound = AudioSegment.from_file("out"+str(i)+".wav", format="wav")

        start_trim = detect_leading_silence(sound)
        end_trim = detect_leading_silence(sound.reverse())

        duration = len(sound)    
        trimmed_sound = sound[start_trim:duration-end_trim]
        os.remove("out"+str(i)+".wav")
        trimmed_sound.export("out"+str(i)+".wav", format="wav")
        if os.path.getsize("out"+str(i)+".wav") < 15000:
            os.remove("out"+str(i)+".wav")
           # command = "autosub -S en -D en out"+str(i)+".wav"
       # subprocess.call(command, shell=True)

from pydub import AudioSegment
import glob,os
def detect_leading_silence(sound, silence_threshold=-50.0, chunk_size=10):
    '''
    sound is a pydub.AudioSegment
    silence_threshold in dB
    chunk_size in ms

    iterate over chunks until you find the first one with sound
    '''
    trim_ms = 0 # ms

    assert chunk_size > 0 # to avoid infinite loop
    while sound[trim_ms:trim_ms+chunk_size].dBFS < silence_threshold and trim_ms < len(sound):
        trim_ms += chunk_size

    return trim_ms

        
        
        



In [ ]:
split_autosub("daddji.mp4","https://www.youtube.com/watch?v=8G5GYoBwIMY")

In [ ]:
for wav in glob.glob(os.path.abspath("*.wav")):
    try:
        os.remove(wav.split('.wav')[0]+".srt")
    except:
        pass
    try:
        command = "autosub2 -S en -D en "+wav
        subprocess.call(command, shell=True)
        try:
            if os.path.getsize(wav.split(".wav")[0]+".srt") < 1:
                print (os.path.getsize(wav.split(".wav")[0]+".srt"))
                os.remove(wav.split('.wav')[0]+".srt")
                command = "autosub -S en -D en "+wav
                subprocess.call(command, shell=True)
        except:
            pass
    except:
        try:
            command = "autosub -S en -D en "+wav
            subprocess.call(command, shell=True)
        except: 
            pass




In [ ]:
import pandas as pd
import re
out=[]
col=["wav_filename","wav_filesize","transcript"]
for wav in glob.glob(os.path.abspath("*.wav")):
    txt1 =""
    txt = wav.split('.wav')[0]+".srt"
    print (os.stat(txt).st_size)
    try:
        if os.stat(txt).st_size != 0:
            subs = pysrt.open(txt, encoding='iso-8859-1')
            for i in range(0,len(subs)):
                txt1 = txt1 + subs[i].text + " "
        path = os.path.abspath(wav)
        r = re.sub("[^a-zA-Z]+", " ", txt1)
        row = [path,os.stat(wav).st_size,r.lower()]
        out.append(row)
    except:
        print ((wav))
    



In [ ]:
data=pd.DataFrame(out)
data.columns=col

In [ ]:
data['wav_filename'] = data['wav_filename'].str.replace("/Users/devansh/Desktop/yotube_cardekho/dadji_vid/","/home/devanshg/DeepSpeech/data/dadji_vid/")



In [ ]:
data_train=data.iloc[0:165]
data_test = data.iloc[165:184]
data_dev = data.iloc[184:]

In [ ]:
data_train.to_csv("dadji_train.csv",index=False,encoding='utf-8')
data_test.to_csv("dadji_test.csv",index=False,encoding='utf-8')
data_dev.to_csv("dadji_dev.csv",index=False,encoding='utf-8')

In [ ]:
data

In [32]:
from pydub import AudioSegment
import glob,os
import pandas as pd
import re
out=[]
col=["wav_filename","wav_filesize","transcript"]
def detect_leading_silence(sound, silence_threshold=-50.0, chunk_size=10):
    '''
    sound is a pydub.AudioSegment
    silence_threshold in dB
    chunk_size in ms

    iterate over chunks until you find the first one with sound
    '''
    trim_ms = 0 # ms

    assert chunk_size > 0 # to avoid infinite loop
    while sound[trim_ms:trim_ms+chunk_size].dBFS < silence_threshold and trim_ms < len(sound):
        trim_ms += chunk_size

    return trim_ms
subs = pysrt.open('dadji_you.srt', encoding='iso-8859-1')
for i in range(0,len(subs)):
    start =(subs[i].start.minutes*60 + subs[i].start.seconds+subs[i].start.milliseconds*.001) 
    end = (subs[i+1].start.minutes*60 + subs[i+1].start.seconds+subs[i+1].start.milliseconds*.001)
    print (start,"-->",end)
    command = "ffmpeg -i daddji.wav -ss "+str(start)+" -to "+str(end)+" -acodec copy out"+str(i)+".wav"
    subprocess.call(command, shell=True)
    sound = AudioSegment.from_file("out"+str(i)+".wav", format="wav")

    start_trim = detect_leading_silence(sound)
    end_trim = detect_leading_silence(sound.reverse())

    duration = len(sound)    
    trimmed_sound = sound[start_trim:duration-end_trim]
    trimmed_sound.export("out"+str(i)+".wav", format="wav")
    wav = "out"+str(i)+".wav"
    row = [os.path.abspath(wav),os.stat(wav).st_size,subs[i].text.lower()]
    print (row)
    out.append(row)



    


58.09 --> 64.82
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out0.wav', 190892, u'namastey ji i felt that i left out']
64.82 --> 69.62
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out1.wav', 151596, u'something yesterday i said complete that']
69.62 --> 76.55
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out2.wav', 225324, u'aspect of conducting a lifestyle in such']
76.55 --> 82.04
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out3.wav', 172076, u'a way in order to avoid cleaning in the']
82.04 --> 91.25
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out4.wav', 294956, u'union imagine if you had to hear she']
91.25 --> 93.979
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out5.wav', 89516, u'happened to hear that master has to do']
93.979 --> 99.11
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out6.wav', 163884, u'cleaning for himself every evening would']
99.11 --> 108.369
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/ou

[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out65.wav', 106540, u'exchange thoughts through food also we']
367.12 --> 369.159
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out66.wav', 65580, u'exchange lot of things we exchange']
369.159 --> 375.49
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out67.wav', 200748, u'nutrients we absorb nutrients the state']
375.49 --> 378.4
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out68.wav', 94252, u'of mind while we are consuming food is']
378.4 --> 380.919
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out69.wav', 77868, u'also very important the way state of']
380.919 --> 383.289
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out70.wav', 77868, u'mind before meditation during meditation']
383.289 --> 385.93
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out71.wav', 86060, u'and after meditation is so critical to']
385.93 --> 391.03
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out72

[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out129.wav', 90156, u'such a state of mind as if master']
621.69 --> 626.91
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out130.wav', 167980, u'himself is having it and when food is']
626.91 --> 628.92
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out131.wav', 65580, u'prepared with this idea that it is a']
628.92 --> 631.04
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out132.wav', 63340, u"master it's been to consume this food"]
631.04 --> 634.41
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out133.wav', 91116, u'then can you imagine that charge of that']
634.41 --> 640.35
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out134.wav', 188460, u'food bob woody says that when you come']
640.35 --> 643.83
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out135.wav', 110636, u'when it is like this all kinds of']
643.83 --> 649.89
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out136.wav

[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out192.wav', 53612, u'consciousness also become divine']
936.23 --> 940.52
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out193.wav', 135212, u'consciousness now what is the difference']
940.52 --> 942.74
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out194.wav', 73772, u'between our consciousness and divine']
942.74 --> 946.43
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out195.wav', 114092, u'consciousness there is also a huge']
946.43 --> 954.11
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out196.wav', 245804, u'spectrum see to give an example person']
954.11 --> 959.3
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out197.wav', 167980, u'may be sleeping in a room you shake him']
959.3 --> 964.22
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out198.wav', 155692, u'up there are certain individuals the']
964.22 --> 967.7
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out199.wav

[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out255.wav', 82732, u'if you kill a goat or a donkey or a cow']
1191.29 --> 1193.57
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out256.wav', 73772, u'and eat that the subtle body is going to']
1193.57 --> 1197.29
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out257.wav', 118828, u'affect you it becomes part of your']
1197.29 --> 1199.39
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out258.wav', 69356, u'subtle body that subtle body of an']
1199.39 --> 1206.82
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out259.wav', 146412, u'animal becomes part of your subtle body']
1206.82 --> 1209.68
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out260.wav', 64556, u'when we consume vegetables and all that']
1209.68 --> 1212.83
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out261.wav', 102124, u'that subtle body is not so labeled and']
1212.83 --> 1222.78
[u'/Users/devansh/Desktop/yotube_cardekho

...

[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out381.wav', 94252, u"other person you're not creating"]
1658.279 --> 1661.759
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out382.wav', 110636, u'impression on yourself but not on others']
1661.759 --> 1662.84
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out383.wav', 32812, u'as well']
1662.84 --> 1665.059
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out384.wav', 73772, u"your existence doesn't create ripples"]
1665.059 --> 1669.499
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out385.wav', 139308, u'anymore if at all it sends out the waves']
1669.499 --> 1678.049
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out386.wav', 274476, u'of purity love because creating']
1678.049 --> 1682.359
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out387.wav', 130348, u'impressions on others also is a problem']
1682.359 --> 1685.519
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out388.wav'

[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out442.wav', 200748, u'because of our in actions in actions are']
1917.559 --> 1921.33
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out443.wav', 122924, u'often remembered before going to bed and']
1921.33 --> 1925.0
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out444.wav', 118828, u'sometimes if they are so important it']
1925.0 --> 1926.47
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out445.wav', 45100, u'will wake you up in the middle of night']
1926.47 --> 1929.5
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out446.wav', 98348, u"and say oh you didn't do this you forgot"]
1929.5 --> 1935.5
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out447.wav', 21996, u'to do this']
1935.5 --> 1939.85
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out448.wav', 130668, u'and second category of thoughts that']
1939.85 --> 1946.3
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out449.wav', 

[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out505.wav', 77868, u'of player our star with different terms']
2152.44 --> 2155.89
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out506.wav', 110636, u'in different words osmosis you are in']
2155.89 --> 2163.26
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out507.wav', 237612, u'osmosis and when we established this']
2163.26 --> 2169.78
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out508.wav', 208940, u'constancy of maya asta then nothing']
2169.78 --> 2176.62
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out509.wav', 217132, u'enters us nothing absolutely we lose out']
2176.62 --> 2183.07
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out510.wav', 208940, u'on the purity we absorb something from']
2183.07 --> 2188.14
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out511.wav', 159788, u'above spread the purity nothing comes']
2188.14 --> 2190.69
[u'/Users/devansh/Desktop/yotube_cardek

[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out566.wav', 184364, u'reduced from there considerably but at']
2389.63 --> 2391.34
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out567.wav', 57388, u'the same time since we are living in']
2391.34 --> 2394.67
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out568.wav', 106540, u'this world you have to learn how to play']
2394.67 --> 2399.47
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out569.wav', 151596, u"with two balls so it's easy to take a"]
2399.47 --> 2401.9
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out570.wav', 77868, u'sannyas and have only one vertical']
2401.9 --> 2405.26
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out571.wav', 110636, u'connection but for us it cannot happen']
2405.26 --> 2412.46
[u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out572.wav', 229420, u"it's unnatural in fact we can make our"]
2412.46 --> 2417.16
[u'/Users/devansh/Desktop/yotube_cardekho/d

IndexError: list index out of range

In [40]:
data=pd.DataFrame(out)
data.columns=col
data['wav_filename'] = data['wav_filename'].str.replace("/Users/devansh/Desktop/yotube_cardekho/dadji_vid/","/home/devanshg/DeepSpeech/data/dadji_vid2/2/")
data_train=data.iloc[0:472]
data_test = data.iloc[472:531]
data_dev = data.iloc[531:589]
data_train.to_csv("dadji_train.csv",index=False,encoding='utf-8')
data_test.to_csv("dadji_test.csv",index=False,encoding='utf-8')
data_dev.to_csv("dadji_dev.csv",index=False,encoding='utf-8')


In [30]:
i=7
start =(subs[i].start.minutes*60 + subs[i].start.seconds+subs[i].start.milliseconds*.001) 
end = (subs[i].end.minutes*60 + subs[i].end.seconds+subs[i].end.milliseconds*.001)

In [39]:
data_train

,wav_filename,wav_filesize,transcript
0,/home/devanshg/DeepSpeech/data/dadji_vid2/2/ou...,190892,namastey ji i felt that i left out
1,/home/devanshg/DeepSpeech/data/dadji_vid2/2/ou...,151596,something yesterday i said complete that
2,/home/devanshg/DeepSpeech/data/dadji_vid2/2/ou...,225324,aspect of conducting a lifestyle in such
3,/home/devanshg/DeepSpeech/data/dadji_vid2/2/ou...,172076,a way in order to avoid cleaning in the
4,/home/devanshg/DeepSpeech/data/dadji_vid2/2/ou...,294956,union imagine if you had to hear she
5,/home/devanshg/DeepSpeech/data/dadji_vid2/2/ou...,89516,happened to hear that master has to do
6,/home/devanshg/DeepSpeech/data/dadji_vid2/2/ou...,163884,cleaning for himself every evening would
7,/home/devanshg/DeepSpeech/data/dadji_vid2/2/ou...,294956,you accept such a master the fact that
8,/home/devanshg/DeepSpeech/data/dadji_vid2/2/ou...,167980,we want to become like master who does
9,/home/devanshg/DeepSpeech/data/dadji_vid2/2/ou...,77868,not need to clean himself like us every


In [20]:
end

69.62

In [26]:
os.path.abspath("out0.wav")

u'/Users/devansh/Desktop/yotube_cardekho/dadji_vid/2/out0.wav'